In [1]:
#importing statements, libraries we use
import re
import pandas as pd
import string
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from textblob import TextBlob

In [2]:
path = '/mnt/c/Users/Zackw/Downloads/2019 MCP Cohort 2/data_clean.pkl'
data = pd.read_pickle(path)

pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

data['polarity'] = data['Text'].apply(pol)
data['subjectivity'] = data['Text'].apply(sub)
polarest = data[data['polarity'] == data['polarity'].max()]
subjlest = data[data['subjectivity'] == data['subjectivity'].min()]
print(data.polarity.describe())
print()
print(data.subjectivity.describe())


count    78.000000
mean      0.173725
std       0.053689
min       0.061553
25%       0.142741
50%       0.174997
75%       0.212074
max       0.360078
Name: polarity, dtype: float64

count    78.000000
mean      0.467621
std       0.049202
min       0.335365
25%       0.437831
50%       0.466081
75%       0.502138
max       0.597603
Name: subjectivity, dtype: float64


In [3]:
print(polarest)
print(subjlest)

                                                                     Text  \
Hiwet Habtemariam.docx  managerial certificate final assessment a succ...   

                        polarity  subjectivity  
Hiwet Habtemariam.docx  0.360078      0.552495  
                                                                         Text  \
Henri-Claude Voltaire.docx   unfpa managerial certification programmefinal...   

                            polarity  subjectivity  
Henri-Claude Voltaire.docx   0.10096      0.335365  


In [4]:
def sect(text, num):
    l = str.split(text)
    arr = np.array_split(l, num)
    #for each elem in arr, we want to convert to a single string chunk
    res = []
    for splits in arr:
        r = combine(splits)
        score = round(pol(r), 3)
        res.append(score)
    
    return res

def combine(l):
    #combines list of strings into chunk
    start = " "
    return start.join(l)

def pol_scan(dat, x):
    data_copy = pd.DataFrame.copy(dat)

    data_copy['pol scores'] = data_copy['Text'].apply(lambda y: sect(y, x))

    return data_copy

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def get_extremes(ds, num):
    data2 = pol_scan(data, 3)
    scores = data2['pol scores']
    agg = []
    for i in scores:
        agg += i
    list.sort(agg)
    mins = agg[0: num]
    maxs = agg[-1 * num:]
    sources = []
    for j, source in enumerate(data2['pol scores']):
        if len(intersection(mins, source)) != 0:
            sources.append(j) #SHOULD NOT BE J, BUT THE SOURCE CORRESPONDING TO J
        if len(intersection(maxs, source)) != 0:
            sources.append(j)
    return data2.iloc[sources]

In [5]:
get_extremes(data, 5)

,Text,polarity,subjectivity,pol scores
Bakhtior Kadyrov.docx,final paper i past what insightsnew behaviors ...,0.075884,0.459873,"[0.106, 0.114, -0.027]"
Bobir Djuraev.docx,past what insightsnew behaviours have you alre...,0.151307,0.412777,"[0.025, 0.275, -0.042]"
Esther Tusabomu Mulumba.docx,i directly supervise six colleagues during thi...,0.212241,0.488001,"[0.153, 0.374, 0.15]"
Final Paper_Nelida Rodrigues.docx,final paper – nelida rodriguesit has been chal...,0.321223,0.490828,"[0.306, 0.384, 0.275]"
Giulio Perna.docx,i think the module of change management is ver...,0.083852,0.407117,"[0.131, 0.103, 0.005]"
Hiwet Habtemariam.docx,managerial certificate final assessment a succ...,0.360078,0.552495,"[0.253, 0.354, 0.48]"
Lela Bakradze.docx,final assignmentby lela bakradzei find people ...,0.231581,0.532045,"[0.354, 0.235, 0.122]"
Lina Mousa.docx,final paper past what insightsnew behaviours h...,0.095623,0.476325,"[-0.049, 0.18, 0.179]"
Management Training Programme_Farid Babayev_2019.docx,management training programme template of the ...,0.190417,0.459418,"[0.002, 0.225, 0.336]"
Vilaykham LASASIMMA Final paper.docx,vilaykham lasasimma final paper – procurement ...,0.254693,0.547487,"[0.109, 0.224, 0.392]"


Conclusions gathered: 
"Negative" language seems to be when the manager says they "will continue" to do things, or "reiterate". Basically if the module training taught them to do things they already knew. They responded quite negatively to rehashing material, as they hint it is a waste of time.
Maybe tell them to provide a specific experiment for their branch- "which I did through the establishment of some formal and other informal ways of communication with him, namely weekly call and monthly meetings as well as use of some informal like using WhatsApp ", " Additionally, to that, I intend to challenge the staff to explore and adhere to some very interesting initiatives, such us HQ initiative on Team Coaches and Team Health. For this I am trying first of all to help to establish a Peer Support Group this year and following the staff retreat, moving to something more comprehensive in 2020"